In [50]:
import random
n=7
V = [(random.uniform(5,75),random.uniform(5,55)) for i in range(n)]
import matplotlib.pyplot as plt
alpha=0.5

In [51]:
V

[(49.823525164394056, 46.90980814856092),
 (59.76933517229901, 36.98378053511337),
 (48.78474812373823, 48.3637507648895),
 (22.58008463241432, 23.60386194833535),
 (61.29696582140142, 31.50133132711721),
 (22.09613375703176, 41.10305666284079),
 (57.09925381239966, 48.31321181905019)]

In [52]:
def t_cost(s,t): return ((s[0]-t[0])**2+(s[1]-t[1])**2)**(0.5)
def d_cost(s,t): return alpha*(((s[0]-t[0])**2+(s[1]-t[1])**2)**(0.5))

In [53]:
#First Pass
import itertools
n=len(V)
Dt={}
truck={}
for i in range(1,n+1):
    for a in itertools.combinations(range(n),i):
        S=frozenset(a)
        for v in range(n):#始点
            for w in S:#終点
                Dt[(S,v,w)]=float('inf')#無限大に初期化
for i in range(1,n+1):
    for a in itertools.combinations(range(n),i):
        S=frozenset(a)
        for v in range(n):
            for w in S:
                if i==1:
                    Dt[(S,v,w)]=t_cost(V[v],V[w])
                    truck[(S,v,w)]=[w]
                else:
                    Sw=S-set([w])
                    for u in Sw:
                        z=Dt[(Sw,v,u)]+t_cost(V[u],V[w])
                        p=truck[(Sw,v,u)]+[w]
                        if z<Dt[(S,v,w)]:
                            Dt[(S,v,w)]=z
                            truck[(S,v,w)]=p


                            

In [54]:
#Second Pass
import itertools
n=len(V)
Dop={}
operation={}
for i in range(1,n+1):
    for a in itertools.combinations(range(n),i):
        S=frozenset(a)
        for v in range(n):#始点
            for w in S:#終点
                Dop[(S,v,w)]=Dt[(S,v,w)]#初期値
                
for i in range(1,n+1):
    for a in itertools.combinations(range(n),i):
        S=frozenset(a)
        for v in range(n):
            for w in S:
                if i==1:
                    Dop[(S,v,w)]=Dt[(S,v,w)]
                    operation[(S,v,w)]=truck[(S,v,w)]
                else:
                    Svw=S-set([v])-set([w])
                    for d in Svw:
                        drone_cost=d_cost(V[v],V[d])+d_cost(V[d],V[w])
                        Sd=S-set([d])
                        z=max(drone_cost,Dt[(Sd,v,w)])
                        if z<Dop[(S,v,w)]:
                            Dop[(S,v,w)]=z
                            operation[(S,v,w)]=truck[(Sd,v,w)]+[[d]]
                 

                
                    
                        
                                

    #Dop(S,v,w)とはv\inS,w\inSでありvからスタートしてSの全ての点を通ってwで終わることを表す

In [55]:
for i in range(1,n+1):
    for a in itertools.combinations(range(n),i):
        S=frozenset(a)
        for j in range(n):
            if (S,j,j) in Dop:
                del Dop[(S,j,j)]

In [56]:
# for i in range(n):
#     del Dop[(frozenset({i}),i,i)]

In [57]:
import pulp
import itertools
import time
prob=pulp.LpProblem("drone_salesman")

x=[pulp.LpVariable('x{}'.format(i),cat=pulp.LpBinary) for i in range(len(Dop))]
y=[pulp.LpVariable('y{}'.format(i),cat=pulp.LpBinary) for i in range(n)]
Dopindex=[]
for i in Dop:
    Dopindex.append(i)
O=[[] for i in range(n)]
Ostart=[[] for i in range(n)]
Oend=[[] for i in range(n)]
for i in range(n):
    for j in range(len(Dop)):
        if Dopindex[j][2]==i:
            Oend[i].append(j)
        if Dopindex[j][1]==i:
            Ostart[i].append(j)
        if i in Dopindex[j][0] or Dopindex[j][1]==i:
            O[i].append(j)

prob += pulp.lpSum(x[i]*Dop[Dopindex[i]] for i in range(len(Dop)))

for i in range(n):
    prob += pulp.lpSum(x[j] for j in O[i])>=1#全てのノードをカバー
    prob += pulp.lpSum(x[j] for j in Ostart[i])<=n*y[i]#vから始まるオペレーションがあるならyは1になる
    prob += pulp.lpSum(x[j] for j in Ostart[i])==pulp.lpSum(x[j] for j in Oend[i])#流量保存
    

for i in range(1,n):
    for a in itertools.combinations(range(n),i):
        S=frozenset(a)
        OS=[]
        for v in S:
            for j in Oend[v]:
                    OS.append(j)
        for v in S:
            prob += pulp.lpSum(x[s] for s in OS)>=y[v]
            
prob += pulp.lpSum(x[s] for s in Oend[0])>=1
prob += y[0]==1
            
# for i in range(1,n):#部分巡回路を削除
#     for a in itertools.combinations(range(n),i):
#         S=frozenset(a)
#         OSC=[]#エンドノードがSで終わらないオペレーションのリスト
#         for v in S:
#             for j in Ostart[v]:
#                 if Dopindex[j][2] not in S:
#                     OSC.append(j)
#         for v in S:
#             prob += pulp.lpSum(x[s] for s in OSC)>=y[v]    
solver = pulp.PULP_CBC_CMD()
time_start = time.perf_counter()    
prob.solve(solver)
time_stop = time.perf_counter()
print(pulp.LpStatus[prob.status]) 
print("objective: {}".format(pulp.value(prob.objective)))
for i in range(len(x)):
    if pulp.value(x[i])==1.0:
        print(Dopindex[i])
print(f"計算時間 = {time_stop - time_start:.3f} (秒)")

Optimal
objective: 56.04352209285884
(frozenset({2}), 0, 2)
(frozenset({4}), 1, 4)
(frozenset({0, 6}), 2, 0)
(frozenset({1, 4}), 4, 1)
(frozenset({3, 5}), 3, 5)
(frozenset({3, 5}), 5, 3)
計算時間 = 5.710 (秒)


In [27]:
Dop[(frozenset({6,9,3}),6,3)]+Dop[(frozenset({3,2,5}),3,5)]+Dop[(frozenset({5,0,8,4}),5,4)]+Dop[(frozenset({4,7,1,6}),4,6)]

114.3651134100759

In [293]:
for i in range(n):
    print(i)

0
1
2
3
4
5


In [294]:
for i in range(1,n):
    print(i)

1
2
3
4
5
